# Oil Price Technical Analysis with Python

**Technical indicators, trend analysis, and visualization**

## What You'll Learn
- Fetch historical oil price data using [OilPriceAPI](https://oilpriceapi.com)
- Calculate technical indicators (SMA, EMA, RSI)
- Analyze volatility and trends
- Identify trading signals

**Get your free API key**: [oilpriceapi.com/auth/signup](https://oilpriceapi.com/auth/signup)

**GitHub**: [OilpriceAPI/kaggle-notebooks](https://github.com/OilpriceAPI/kaggle-notebooks)

In [ ]:
!pip install oilpriceapi pandas numpy matplotlib seaborn -q

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from oilpriceapi import OilPriceAPI

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 8)
print("✅ Libraries loaded")

## Initialize Client

In [ ]:
# Get API key
try:
    from kaggle_secrets import UserSecretsClient
    api_key = UserSecretsClient().get_secret("OILPRICEAPI_KEY")
except:
    api_key = os.environ.get('OILPRICEAPI_KEY')

# Initialize client with longer timeout for Kaggle environment
client = OilPriceAPI(api_key=api_key, timeout=120)
print("✅ Client initialized")

## Fetch Data

In [ ]:
# Get past year of Brent Crude data
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

print("Fetching Brent Crude data...")
df = client.historical.to_dataframe(
    commodity='BRENT_CRUDE_USD',
    start=start_date,
    end=end_date,
    interval='daily'
)

# Rename for clarity
df = df[['value']].rename(columns={'value': 'Price'})

print(f"✅ Loaded {len(df)} days of data")
print(f"📅 Date range: {df.index.min().date()} to {df.index.max().date()}")
print(f"💰 Price range: ${df['Price'].min():.2f} - ${df['Price'].max():.2f}")

## Technical Indicators

In [ ]:
# Simple Moving Averages
df['SMA_20'] = df['Price'].rolling(window=20).mean()
df['SMA_50'] = df['Price'].rolling(window=50).mean()

# Exponential Moving Average
df['EMA_20'] = df['Price'].ewm(span=20, adjust=False).mean()

# RSI (Relative Strength Index)
delta = df['Price'].diff()

# Fill the first row (NaN) with 0 to avoid comparison warnings
delta = delta.fillna(0)

gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
df['RSI'] = 100 - (100 / (1 + rs))

# Daily Returns
df['Returns'] = df['Price'].pct_change() * 100

print("✅ Technical indicators calculated")

## Visualization: Price with Moving Averages

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

# Plot 1: Price with MAs
ax1.plot(df.index, df['Price'], label='Price', linewidth=2, alpha=0.8, color='black')
ax1.plot(df.index, df['SMA_20'], label='SMA 20', linewidth=2, linestyle='--', color='blue')
ax1.plot(df.index, df['SMA_50'], label='SMA 50', linewidth=2, linestyle='--', color='red')
ax1.plot(df.index, df['EMA_20'], label='EMA 20', linewidth=2, linestyle=':', color='green')

# Identify crossovers
df['Signal'] = 0
df.loc[df['SMA_20'] > df['SMA_50'], 'Signal'] = 1
df.loc[df['SMA_20'] < df['SMA_50'], 'Signal'] = -1

# Calculate crossovers - dropna() removes the first row which is often a false positive
crossovers = df[df['Signal'].diff() != 0].dropna()

bullish = crossovers[crossovers['Signal'] == 1]
bearish = crossovers[crossovers['Signal'] == -1]

ax1.scatter(bullish.index, bullish['Price'], color='green', marker='^', s=200, 
           label='Bullish Crossover', zorder=5)
ax1.scatter(bearish.index, bearish['Price'], color='red', marker='v', s=200, 
           label='Bearish Crossover', zorder=5)

ax1.set_ylabel('Price (USD/barrel)', fontsize=12)
ax1.set_title('Brent Crude with Moving Averages & Crossovers', fontsize=14, fontweight='bold')
ax1.legend(loc='upper left', fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: RSI
ax2.plot(df.index, df['RSI'], linewidth=2, color='purple')
ax2.axhline(y=70, color='red', linestyle='--', alpha=0.7, label='Overbought (70)')
ax2.axhline(y=30, color='green', linestyle='--', alpha=0.7, label='Oversold (30)')
ax2.fill_between(df.index, 30, 70, alpha=0.1, color='gray')
ax2.set_ylabel('RSI', fontsize=12)
ax2.set_xlabel('Date', fontsize=12)
ax2.set_title('Relative Strength Index (RSI)', fontsize=14, fontweight='bold')
ax2.legend(loc='upper left', fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_ylim([0, 100])

plt.tight_layout()
plt.show()

print(f"\n📊 Current RSI: {df['RSI'].iloc[-1]:.2f}")
if df['RSI'].iloc[-1] > 70:
    print("⚠️ Market is OVERBOUGHT")
elif df['RSI'].iloc[-1] < 30:
    print("⚠️ Market is OVERSOLD")
else:
    print("✅ Market is in NEUTRAL territory")

## Key Insights

In [ ]:
current_price = df['Price'].iloc[-1]
price_change = df['Price'].iloc[-1] - df['Price'].iloc[0]
price_change_pct = (price_change / df['Price'].iloc[0]) * 100

print("=" * 60)
print("📊 KEY INSIGHTS")
print("=" * 60)
print(f"\n💰 Current Price: ${current_price:.2f}/barrel")
print(f"📈 YoY Change: ${price_change:+.2f} ({price_change_pct:+.2f}%)")
print(f"📊 Volatility: {df['Returns'].std():.2f}% (daily)")
print(f"\n🎯 Trading Signals:")
print(f"  - Bullish Crossovers: {len(bullish)}")
print(f"  - Bearish Crossovers: {len(bearish)}")
print(f"  - Current RSI: {df['RSI'].iloc[-1]:.2f}")
print("\n" + "=" * 60)

## Resources

- **API Documentation**: [docs.oilpriceapi.com](https://docs.oilpriceapi.com)
- **Python SDK**: [github.com/oilpriceapi/python-sdk](https://github.com/oilpriceapi/python-sdk)
- **Get Free API Key**: [oilpriceapi.com/auth/signup](https://oilpriceapi.com/auth/signup)
- **This Notebook**: [github.com/OilpriceAPI/kaggle-notebooks](https://github.com/OilpriceAPI/kaggle-notebooks)

---

*Powered by [OilPriceAPI](https://oilpriceapi.com) - Real-time commodity price data*